In [1]:
# Import essential modules
import numpy as np
import pandas as pd
import itertools; import joblib; import time
from scipy import linalg

In [2]:
# Set pandas options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Import custom module
import module.predict_by_logistic_reservoir_v2 as lrt

In [4]:
import os; output_dir = "02_LogisticRC_TSpredOut";

In [5]:
# ----------------------------------------------- #
# Prepare data and set parameters
# ----------------------------------------------- #
# Load time series
lorenz = pd.read_csv('./data/LorenzTS_tau10.csv')

# Prepare output dataframe
output_all_df = pd.DataFrame()

# Set target variable
target_var, target_ts = 'x', lorenz[91:]

# Netrowk parameters
num_reservoir_nodes = 2 # Equals to the number of variables in the logistic map
#num_input_nodes = 1 # Input is a scalar value
#num_output_nodes = 1 # Output is a scalar vlue

# Specify the test fraction
test_fraction = 0.2

In [6]:
# Paratemers of the model:
rx0 = 3.0
ry0 = 2.7
bxy0 = -0.2
byx0 = 0.2

In [7]:
# ----------------------------------------------- #
# Search w_in_sparsity and w_in_strength
# ----------------------------------------------- #
# For parallel analysis
def reservoir_computing_parallel(w_in_sparsity = 0.1, w_in_strength = 0.1,
                                 rx = rx0, ry = ry0, bxy = bxy0, byx = byx0):
  # Step.1: Initialize class "SimplexReservoir"
  par_lrc = lrt.LogisticReservoir(network_name = "Logistic Map")
  # Step.2: Prepare target data
  par_lrc.prepare_target_data(target_ts, target_var, test_fraction)
  # Step.3: Initialize reservoir
  par_lrc.initialize_reservoir(num_reservoir_nodes = 2, w_in_sparsity = w_in_sparsity, w_in_strength = w_in_strength)
  # Step.4: Training reservoir
  par_lrc.compute_reservoir_state(rx = rx, ry = ry, bxy = bxy, byx = byx)
  # Step.5: Learn weights by ridge regression
  par_lrc.learn_model(ridge_lambda = 0.05, washout_fraction = 0.05)
  # Step.6: Predict test data
  par_lrc.predict()
  # Step.7: Summarize stats
  par_lrc.summarize_stat()
  return par_lrc.result_summary_df
# ------------------------------------------------------------------------------------------ #

In [9]:
# Parameter search
WinSp_range = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
WinSt_range = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

par_rc1 = joblib.Parallel(n_jobs=-3, verbose = 10)([joblib.delayed(reservoir_computing_parallel)
                                    (w_in_sparsity = x, w_in_strength = y) for x, y in itertools.product(WinSp_range, WinSt_range)])
output_all_df1 = par_rc1[0]
for j in range(1,len(par_rc1)): output_all_df1 = output_all_df1.append(par_rc1[j])
output_all_df1 = output_all_df1.reset_index(drop = True)
wspst_min = output_all_df1.loc[output_all_df1['NMSE_test'].min() == output_all_df1['NMSE_test']]
w_in_sp_best = float(np.array(wspst_min["Win_sparsity"])[0])
w_in_st_best = float(np.array(wspst_min["Win_strength"])[0])

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done   1 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-3)]: Done  12 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-3)]: Done  25 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-3)]: Done  38 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-3)]: Done  53 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-3)]: Batch computation too fast (0.1763s.) Setting batch_size=2.
[Parallel(n_jobs=-3)]: Done  68 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-3)]: Done  85 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-3)]: Done 118 out of 160 | elapsed:    3.6s remaining:    1.2s
[Parallel(n_jobs=-3)]: Done 135 out of 160 | elapsed:    3.6s remaining:    0.6s
[Parallel(n_jobs=-3)]: Done 152 out of 160 | elapsed:    3.7s remaining:    0.1s
[Parallel(n_jobs=-3)]: Done 160 out of 160 | elapsed:    3.9s finished
C:\Users\alpez\AppData\Local\Temp\ipykernel_19344\56713534.py:8: FutureWarning: The frame.ap

In [10]:
# Perform the best results
lrc = lrt.LogisticReservoir(network_name = "Logistic Map")
lrc.prepare_target_data(target_ts, target_var, test_fraction)
lrc.initialize_reservoir(num_reservoir_nodes, w_in_sparsity = w_in_sp_best, w_in_strength = w_in_st_best)
lrc.compute_reservoir_state(rx = rx0, ry = ry0, bxy = bxy0, byx = byx0)
lrc.learn_model(ridge_lambda = 0.05)
lrc.predict()
lrc.summarize_stat()
lrc.result_summary_df

,network,rho_train,rho_test,RMSE_train,RMSE_test,NMSE_train,NMSE_test,rx,ry,bxy,byx,num_nodes,Win_strength,Win_sparsity,data_size,test_fraction,washout_data,training_time,learnmodel_time,testing_time
0,Logistic Map,0.8093,0.7881,0.135,0.1396,0.0568287,0.0632242,3.0,2.7,-0.2,0.2,2,0.7,0.5,900,0.2,0,0.02,0.03,0.0


In [11]:
# Save result to pickel
joblib.dump(lrc, "%s/LR_PredLorenzMOD.jb" % output_dir, compress=3)

['02_LogisticRC_TSpredOut/LR_PredLorenzMOD.jb']

In [12]:
# ----------------------------------------------- #
# Perform simple ridge regression under the same condition
# ----------------------------------------------- #
pred_wo_reservoir = lrt.LogisticReservoir(network_name = "wo_reservoir")
pred_wo_reservoir.prepare_target_data(target_ts, target_var, test_fraction)
pred_wo_reservoir.learn_model_wo_reservoir(ridge_lambda = 0.05)
pred_wo_reservoir.predict_wo_reservoir()

In [14]:
# Save result to pickel
joblib.dump(pred_wo_reservoir, "%s/LR_PredLorenz_wo_reservoir.jb" % output_dir, compress=3)

['02_LogisticRC_TSpredOut/LR_PredLorenz_wo_reservoir.jb']

In [19]:
import sys
print(sys.version)

3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]


In [4]:
import joblib
import pandas as pd
import numpy as np
import math as m

In [2]:
OLS_x = joblib.load(r"C:\Users\alpez\Desktop\Proyectos\Chemputing\RC\ThesisResults\LR_OLS_x.jb")
OLS_y = joblib.load(r"C:\Users\alpez\Desktop\Proyectos\Chemputing\RC\ThesisResults\LR_OLS_y.jb")
OLS_z = joblib.load(r"C:\Users\alpez\Desktop\Proyectos\Chemputing\RC\ThesisResults\LR_OLS_z.jb")
real_x = joblib.load(r"C:\Users\alpez\Desktop\Proyectos\Chemputing\RC\ThesisResults\LR_obs_x.jb")
real_y = joblib.load(r"C:\Users\alpez\Desktop\Proyectos\Chemputing\RC\ThesisResults\LR_obs_y.jb")
real_z = joblib.load(r"C:\Users\alpez\Desktop\Proyectos\Chemputing\RC\ThesisResults\LR_obs_z.jb")

In [3]:
test_x = OLS_x.test_predicted
test_y = OLS_y.test_predicted
test_z = OLS_z.test_predicted
obs_x = real_x.test_true
obs_y = real_y.test_true
obs_z = real_z.test_true

In [7]:
m.sqrt(np.mean((test_z - obs_z)**2))

0.207055956951616